# Everton v. Chelsea - 2024-12-21 match analysis
*This notebook intends to walk through the theory and calculations I am currently using in my model to calculate probability of 1 x 2 result and a table of over / under goal projections. I will break the calculations out into utility functions stored in `../src/utils` for ease of use in future explorations. This is to document the calculations.*

### Instructions:

1. Ensure all cells in all notebooks in `/notebooks` directory have been run.
2. Define `home_side` and `away_side` in first cell. If making a comparison with odds from `03_retrieve_odds.ipynb` also enter matchday.
3. Click 'run all'
4. Probability for 1 x 2 and over / under can be viewed in the last cell
5. *Suggestion:* copy and paste notebook to `../output/< matchday_directory >/` and rename file. IMPORTANT: Update file path in lines 14 and 15 if needed if you make a copy.



### Import League Data & Define Sides

Define 'home_side' and 'away_side'
Ensure correct output



In [2]:
import pandas as pd
from scipy.stats import poisson
import numpy as np

"""
BUG: if copying from `notebooks/03_analysis_template_notebook.ipynb` to a directory in output
for storage, ensure the file path for reading the csv will need to be updated from '../data/processed/home_table.csv'
to '../../data/processed/home_table.csv' if frames are run again.

TLDR
if you are in ./notebooks/03_analysis_tempalte.csv you may run this cell as is. If you are not, check that following
read_csv commands read from the correct directory.
"""
home_dataframe = pd.read_csv('../data/processed/home_table.csv')
away_dataframe = pd.read_csv('../data/processed/away_table.csv')

"""
!!! INPUT TEAMS & MATCHDAY HERE !!!

wrapped in quotes, eg.

home_side = "Bournemouth"
away_side = "West Ham"
matchday = "2024-12-16" # (optional)

If there is a ValueError,
(1) check spelling against the error message. Must be exact.
(2) check capitalization
(3) check that it is wrapped in quotes

!!! INPUT TEAMS & MATCHDAY HERE !!!
"""
home_side = "Everton"
away_side = "Chelsea"
matchday = "2024-12-22" # (optional)

# validate input
clubs = set(home_dataframe['Team'].values)
if home_side not in clubs:
    raise ValueError(f"Invalid entry for 'home_side'. Expected one of {clubs}, but received '{home_side}'")
elif away_side not in clubs:
    raise ValueError(f"Invalid entry for 'away_side'. Expected one of {clubs}, but received '{away_side}'")
else:
    print(f"Ready to calculate probability for {home_side} v. {away_side}")

Ready to calculate probability for Everton v. Chelsea


### Calculate Projected Goals

In [3]:
# 2. grab their gpm_scored and gpm_conceded
home_side_gpm_scored = home_dataframe.loc[
    home_dataframe["Team"] == home_side,
    "gpm_scored"
].values[0]

home_side_gpm_conceded = home_dataframe.loc[
    home_dataframe["Team"] == home_side,
    "gpm_conceded"
].values[0]

print(home_side_gpm_scored, home_side_gpm_conceded)

away_side_gpm_scored = away_dataframe.loc[
    away_dataframe["Team"] == away_side,
    "gpm_scored"
].values[0]

away_side_gpm_conceded = away_dataframe.loc[
    away_dataframe["Team"] == away_side,
    "gpm_conceded"
].values[0]

print(away_side_gpm_scored, away_side_gpm_conceded)

# 3.
epl_home_avg_gpm_scored = round(sum(home_dataframe["gpm_scored"].values) / 20, 2)
epl_home_avg_gpm_conceded = round(sum(home_dataframe["gpm_conceded"].values) / 20, 2)
print(epl_home_avg_gpm_scored, epl_home_avg_gpm_conceded)

# 4. create a ratio (attack and defense rating) of their gpm to the league average gpm
home_attack_rating = home_side_gpm_scored / epl_home_avg_gpm_scored
away_defense_rating = away_side_gpm_conceded / epl_home_avg_gpm_scored
away_attack_rating = away_side_gpm_scored / epl_home_avg_gpm_conceded
home_defense_rating = home_side_gpm_conceded / epl_home_avg_gpm_conceded

print(home_attack_rating, away_defense_rating, away_attack_rating, home_defense_rating)

# home projected goals
home_projected_goals = home_attack_rating * away_defense_rating * epl_home_avg_gpm_scored
# away projected goals
away_projected_goals = away_attack_rating * home_defense_rating * epl_home_avg_gpm_conceded
print("\n ================== projected_goals ==================== \n")
print(f"HOME {home_side} projected goals:", home_projected_goals)
print(f"AWAY {away_side} projected goals: ", away_projected_goals)

1.2857142857142858 1.1428571428571428
2.875 1.25
1.61 1.4
0.7985803016858918 0.7763975155279502 2.053571428571429 0.8163265306122449

 ================== projected_goals ==================== 

HOME Everton projected goals: 0.9982253771073646
AWAY Chelsea projected goals:  2.3469387755102042


### Poisson Distribution of Goal Probabilities

In [4]:
# Poisson probability for home team goals 0 - 8
home_score_prob = []
away_score_prob = []
for i in range(9):
    prob = poisson.pmf(i, home_projected_goals)
    home_score_prob.append(prob)
for i in range(9):
    prob = poisson.pmf(i, away_projected_goals)
    away_score_prob.append(prob)

home_score_prob = np.array(home_score_prob)
away_score_prob = np.array(away_score_prob)
# print(home_score_prob)
# print(away_score_prob)

exact_score_prob = np.outer(away_score_prob, home_score_prob)
print(f"{home_side} v. {away_side}", exact_score_prob)

Everton v. Chelsea [[3.52544276e-02 3.51918643e-02 1.75647060e-02 5.84451175e-03
  1.45853499e-03 2.91189327e-04 4.84454294e-05 6.90849386e-06
  8.62029236e-07]
 [8.27399831e-02 8.25931508e-02 4.12232896e-02 1.37167113e-02
  3.42309232e-03 6.83403524e-04 1.13698457e-04 1.62138121e-05
  2.02312984e-06]
 [9.70928373e-02 9.69205341e-02 4.83742684e-02 1.60961408e-02
  4.01689404e-03 8.01953115e-04 1.33421658e-04 1.90264122e-05
  2.37408093e-06]
 [7.59569816e-02 7.58221866e-02 3.78438154e-02 1.25922190e-02
  3.14246813e-03 6.27378287e-04 1.04377488e-04 1.48846082e-05
  1.85727420e-06]
 [4.45665963e-02 4.44875074e-02 2.22042794e-02 7.38829174e-03
  1.84379508e-03 3.68104607e-04 6.12418934e-05 8.73331602e-06
  1.08972721e-06]
 [2.09190146e-02 2.08818912e-02 1.04224169e-02 3.46797367e-03
  8.65454832e-04 1.72783795e-04 2.87461948e-05 4.09931160e-06
  5.11504608e-07]
 [8.18260775e-03 8.16808671e-03 4.07679572e-03 1.35652031e-03
  3.38528251e-04 6.75854981e-05 1.12442599e-05 1.60347222e-06
  2.0

### Under Market

In [5]:
thresholds = [0.5, 1.5, 2.5, 3.5]
sums = []

for t in thresholds:
    mask = np.add.outer(
        np.arange(
            exact_score_prob.shape[0]
        ),
        np.arange(
            exact_score_prob.shape[1]
        )
    ) <= t
    # print(mask)
    sums.append(exact_score_prob[mask].sum())

# print(sums)
under_market_df = pd.DataFrame({
    "Goals": thresholds,
    "Prob": sums
})
under_market_df["Implied Odds"] = 1 / under_market_df["Prob"]
print("~  ~ under market ~  ~\n", f"{home_side} v. {away_side}\n", under_market_df)

~  ~ under market ~  ~
 Everton v. Chelsea
    Goals      Prob  Implied Odds
0    0.5  0.035254     28.365232
1    1.5  0.153186      6.528000
2    2.5  0.350437      2.853580
3    3.5  0.570382      1.753210


In [6]:
thresholds = [0.5, 1.5, 2.5, 3.5]
sums = []

for t in thresholds:
    mask = np.add.outer(
        np.arange(
            exact_score_prob.shape[0]
        ),
        np.arange(
            exact_score_prob.shape[1]
        )
    ) >= t
    sums.append(exact_score_prob[mask].sum())

# print(sums)
over_market_df = pd.DataFrame({
    "Goals": thresholds,
    "Prob": sums
})

over_market_df["Implied Odds"] = 1 / over_market_df["Prob"]
print("~  ~ over market ~  ~\n", f"{home_side} v. {away_side}\n", over_market_df)

~  ~ over market ~  ~
 Everton v. Chelsea
    Goals      Prob  Implied Odds
0    0.5  0.964006      1.037338
1    1.5  0.846074      1.181929
2    2.5  0.648823      1.541251
3    3.5  0.428878      2.331665


### 1 x 2 Probability

In [7]:
# Poisson probability for home team goals 0 - 8
home_score_prob = []
away_score_prob = []
for i in range(9):
    prob = poisson.pmf(i, home_projected_goals)
    home_score_prob.append(prob)
for i in range(9):
    prob = poisson.pmf(i, away_projected_goals)
    away_score_prob.append(prob)

home_score_prob = np.array(home_score_prob)
away_score_prob = np.array(away_score_prob)
# print(home_score_prob)
# print(away_score_prob)

exact_score_prob = np.outer(away_score_prob, home_score_prob)

# print(exact_score_prob)
home_win_prob = np.sum(np.triu(exact_score_prob, k=1))
away_win_prob = np.sum(np.tril(exact_score_prob, k=-1))
draw_prob = np.trace(exact_score_prob)

# prepare data in readable dataframe
data = {
    "winner": [f"{home_side} (1)", "Draw (x)", f"{away_side} (2)"],
    "prob": [home_win_prob, draw_prob, away_win_prob],
}
result_df = pd.DataFrame(data)
result_df['implied_odds'] = 1 / result_df['prob']
print("      ~  ~ moneyline market ~  ~\n", result_df)

      ~  ~ moneyline market ~  ~
         winner      prob  implied_odds
0  Everton (1)  0.145021      6.895571
1     Draw (x)  0.180842      5.529675
2  Chelsea (2)  0.673397      1.485007


# Full Results Summary:

In [8]:
# helper function to translate discrepency between Understat and Odds-Api naming
def translate(string):

    translations = {
        "Brighton": "Brighton and Hove Albion",
        "Ipswich": "Ipswich Town",
        "Leicester": "Leicester City",
        "Tottenham": "Tottenham Hotspur",
        "West Ham": "West Ham United"
    }

    if string in translations:
        return translations[string]
    return string

home_side, away_side = translate(home_side), translate(away_side)

match_string = f"{home_side} v. {away_side}"

print(f"CALCULATED PROBABILITIES FOR {home_side} v. {away_side}")
print("\n\n        ~ ~ moneyline market ~ ~\n\n", result_df)
print("\n\n          ~ ~ over market ~ ~ \n\n", over_market_df)
print("\n\n          ~ ~ under market ~ ~ \n\n", under_market_df)

try:
    odds_df = pd.read_csv(f"../data/processed/odds/odds_matchday_{matchday}.csv")
    filtered_odds = odds_df.loc[odds_df["match"] == match_string]
    implied = {
        "1": 1 / filtered_odds["1"],
        "x": 1 / filtered_odds["x"],
        "2": 1 / filtered_odds["2"],
        "point": filtered_odds["ovr_und_point"],
        "over": 1 / filtered_odds["over"],
        "under": 1 / filtered_odds["under"]
    }
    implied_odds_df = pd.DataFrame(implied)

    print(f"\n ~~ MyBookie Odds for {match_string} ~~\n",filtered_odds.drop(columns=["match"]))
    print(f"\n ~~ MyBookie Odds IMPLIED for {match_string} ~~\n", implied_odds_df)
except:
    print(f"No data for sportsbook odds on matchday {matchday}. To view against real odds, run `notebooks/03_retrieve_odds.ipynb` ran correctly and ensure matchday is correct in the first cell of this notebook")


CALCULATED PROBABILITIES FOR Everton v. Chelsea


        ~ ~ moneyline market ~ ~

         winner      prob  implied_odds
0  Everton (1)  0.145021      6.895571
1     Draw (x)  0.180842      5.529675
2  Chelsea (2)  0.673397      1.485007


          ~ ~ over market ~ ~ 

    Goals      Prob  Implied Odds
0    0.5  0.964006      1.037338
1    1.5  0.846074      1.181929
2    2.5  0.648823      1.541251
3    3.5  0.428878      2.331665


          ~ ~ under market ~ ~ 

    Goals      Prob  Implied Odds
0    0.5  0.035254     28.365232
1    1.5  0.153186      6.528000
2    2.5  0.350437      2.853580
3    3.5  0.570382      1.753210

 ~~ MyBookie Odds for Everton v. Chelsea ~~
      1    x     2  ovr_und_point  over  under
1  4.9  4.1  1.61            2.5  1.63    2.2

 ~~ MyBookie Odds IMPLIED for Everton v. Chelsea ~~
           1         x         2  point      over     under
1  0.204082  0.243902  0.621118    2.5  0.613497  0.454545


# Conclusion and Pick
SAFE: Chelsea win - Chelsea are reliable lads this season, Everton in bad run of form. The line looks good.

### My Pick:

**Chelsea @ 1.61** -- *2 unit*